In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")


from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import ohe_and_nlp
from model.model_functions import *

from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items.append('')

    return list_of_items


In [6]:
def ohe_api_category(data, column_name):
    """
    Returns OHE columns of api categories
    """
    data[column_name] = data[column_name].apply(get_names_from_dict)

    ohes, mlb = categorical_encoder(data[column_name])

    return ohes


In [5]:
data = pd.read_json('../raw_data/final_api_call')


In [7]:
player_perspective_ohe_columns = ohe_api_category(data, 'player_perspectives')


In [28]:
web_data = pd.read_json('../raw_data/final_data')

# Columns we don't need immediately
cols_to_drop = ['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'platforms',
       'total_reviews', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id']

final_useful_columns = web_data.drop(columns=cols_to_drop)

# descriptions were an onject, we need them as a string!
final_useful_columns['description'] = final_useful_columns['description'].astype(str)


In [10]:
ohe_and_nlp_data = ohe_and_nlp(final_useful_columns, 'genres', 20, 'description', 10)

all_data = pd.concat((ohe_and_nlp_data, player_perspective_ohe_columns), axis=1).fillna(0)


/Users/antonis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [16]:
all_data


,avg_review,total_lists,Adventure,Indie,RPG,Simulator,Puzzle,Strategy,Shooter,Platform,...,22,23,,Auditory,Bird view / Isometric,First person,Side view,Text,Third person,Virtual Reality
0,0.0,0,0,1,0,0,0,0,0,0,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1,1,1,0,0,0,0,0,0,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5,0,0,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0,1,0,0,0,0,0,0,0,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,36,0,0,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37910,3.2,10,0,0,0,0,1,1,0,0,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37911,2.9,9,0,0,0,0,1,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37912,3.5,16,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37913,4.0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
X_train = yeo_johnson_scaling(all_data.drop('avg_review', axis=1), 'total_lists')
y_train = all_data['avg_review']


/Users/antonis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [23]:
X_train


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,-0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.651187,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.528816,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.646256,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37910,1.942935,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37911,1.880808,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37912,2.213377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37913,-0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
model = train_model(X_train= X_train, y_train = y_train)


In [25]:
model_predict(12552, model=model, reference_data=web_data, X_train=X_train)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
12552,The Legend of Zelda: Breath of the Wild,1488499200000,44000,3500,6900,3600,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,131,367,563,1257,2163,5236,6908,13704,/games/the-legend-of-zelda-breath-of-the-wild/,7346
19153,Dishonored,1349740800000,20000,382,4900,1200,"[Arkane Studios, Bethesda Softworks]",4.0,"[Adventure, Puzzle, RPG]","[Windows PC, Xbox 360, PlayStation 3]",...,50,208,379,1135,2131,3753,2218,1877,/games/dishonored/,533
36154,Chicory: A Colorful Tale,1623283200000,1500,174,1300,1200,"[Wishes Ultd., Finji]",4.1,"[Adventure, Indie, Puzzle, RPG]","[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",...,7,16,25,78,145,270,219,238,/games/chicory-a-colorful-tale/,117266
28214,Eternal Darkness: Sanity's Requiem,1024790400000,1300,29,1300,822,"[Gradiente, Silicon Knights]",3.9,"[Adventure, Puzzle]",[Nintendo GameCube],...,8,18,27,92,139,218,129,106,/games/eternal-darkness-sanity-s-requiem/,3907
12302,Dark Souls III: The Fire Fades Edition,1492732800000,1900,61,409,170,"[FromSoftware, Bandai Namco Entertainment]",4.4,"[Adventure, RPG]","[Windows PC, PlayStation 4, Xbox One]",...,0,8,14,34,79,231,344,460,/games/dark-souls-iii-the-fire-fades-edition/,28210
1108,The Legend of Zelda: Tears of the Kingdom,1683849600000,12000,6200,5000,6400,"[Nintendo, Nintendo EPD Production Group No. 3]",4.5,"[Adventure, RPG]",[Nintendo Switch],...,31,78,125,329,494,1235,2211,5530,/games/the-legend-of-zelda-tears-of-the-kingdom/,119388
27494,Star Wars: Knights of the Old Republic,1058227200000,7300,258,3300,942,"[BioWare, LucasArts]",4.2,"[Adventure, RPG]","[Windows PC, Android, Mac, Xbox, iOS, Nintendo...",...,13,66,88,251,466,1083,974,1177,/games/star-wars-knights-of-the-old-republic/,116
17188,Wolfenstein: The New Order,1400544000000,12000,205,3600,965,"[MachineGames, Bethesda Softworks]",3.6,"[Adventure, Puzzle, Shooter]","[Windows PC, Xbox 360, PlayStation 4, PlayStat...",...,52,223,361,1112,1966,2369,754,367,/games/wolfenstein-the-new-order/,2031
8605,A Plague Tale: Innocence,1557792000000,7900,233,3800,1400,"[Asobo Studio, Focus Entertainment]",3.7,"[Adventure, RPG]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,42,135,240,781,1335,1602,553,359,/games/a-plague-tale-innocence/,27316
3149,Horizon Forbidden West,1645142400000,4500,685,3500,3100,"[Sony Interactive Entertainment, Guerrilla Games]",3.8,"[Adventure, RPG]","[PlayStation 4, PlayStation 5]",...,31,100,158,372,553,851,578,391,/games/horizon-forbidden-west/,112874
